# Import required packages

In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pickle import load
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import gc

In [3]:
import tensorflow as tf

In [4]:
import keras.layers as L
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D
from keras.layers import UpSampling2D
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import RepeatVector
from keras.layers import Reshape
from keras.layers import concatenate
from keras.utils import plot_model

# reading files from s3 bucket

In [5]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

In [6]:
import boto3
import sys
import os
import pandas as pd
import csv
import io

s3_client =boto3.client('s3')
s3_bucket_name='recolourise'
s3 = boto3.resource('s3',
                    aws_access_key_id= 'YOUR_ACCESS_KEY_ID',
                    aws_secret_access_key='YOUR_SECRET_ACCESS_KEY')

In [7]:
training_data_uri = 's3://recolorising/processed/'

In [10]:
#Check the contents of the uri
# !aws s3 ls $training_data_uri/

2023-08-26 08:45:03     261991 NLP.png
2023-08-26 08:54:23  451584128 small_set_LAB_3000.npy


In [8]:
from s3fs.core import S3FileSystem
s3 = S3FileSystem()

In [13]:
train_key = 'processed/train.npy'
test_key = 'processed/test.npy'
gray_test_key = 'processed/gray_test.npy'
gray_train_key = 'processed/gray_train.npy'
bucket = 'recolorising'

In [14]:
train = np.load(s3.open('{}/{}'.format(bucket, train_key)))
test = np.load(s3.open('{}/{}'.format(bucket, test_key)))
gray_test = np.load(s3.open('{}/{}'.format(bucket, gray_test_key)))
gray_train = np.load(s3.open('{}/{}'.format(bucket, gray_train_key)))

In [16]:
gray_test.shape

(800, 128, 128)

img = full_img[2,:,:,:]
#img = cv2.cvtColor(np.array(img), cv2.COLOR_Lab2RGB)
plt.imshow(np.array(img).astype('int'))

COnverting RGB back into Lab

Defining Unet architecture

In [18]:
train_iterator = datagen.flow(train, batch_size=64)
test_iterator = datagen.flow(test, batch_size=64)
print('Batches train=%d, test=%d' % (len(train_iterator),
                                     len(test_iterator)))

Batches train=55, test=24


In [21]:
def UNet():
    Xinpt = L.Input([None, None, 1])
    X0 = L.Conv2D(64, (3, 3), padding='same')(Xinpt)
    X0 = L.BatchNormalization()(X0)
    X0 = L.LeakyReLU(alpha=0.2)(X0)    #l,b,64
    X0 = L.Conv2D(64, (3, 3), strides=1, padding='same')(X0)
    X0 = L.BatchNormalization()(X0)
    X0 = L.LeakyReLU(alpha=0.2)(X0)    #l,b,64
    
    X1 = L.MaxPool2D((2, 2), strides=2)(X0)    #l/2,b/2,64
    X1 = L.Conv2D(128, (3, 3), padding='same')(X1)
    X1 = L.BatchNormalization()(X1)
    X1 = L.LeakyReLU(alpha=0.2)(X1)
    X1 = L.Conv2D(128, (3, 3), padding='same')(X1)
    X1 = L.BatchNormalization()(X1)
    X1 = L.LeakyReLU(alpha=0.2)(X1)    #l/2,b/2,128
    
    X2 = L.MaxPool2D((2, 2), strides=2)(X1)    #l/4,b/4,128
    X2 = L.Conv2D(256, (3, 3), padding='same')(X2)
    X2 = L.BatchNormalization()(X2)
    X2 = L.LeakyReLU(alpha=0.2)(X2)
    X2 = L.Conv2D(256, (3, 3), padding='same')(X2)
    X2 = L.BatchNormalization()(X2)
    X2 = L.LeakyReLU(alpha=0.2)(X2)    #l/4,b/4,256
    
    X3 = L.MaxPool2D((2, 2), strides=2)(X2)    #l/8,b/8,256
    X3 = L.Conv2D(512, (3, 3), padding='same')(X3)
    X3 = L.BatchNormalization()(X3)
    X3 = L.LeakyReLU(alpha=0.2)(X3)
    X3 = L.Conv2D(512, (3, 3), padding='same')(X3)
    X3 = L.BatchNormalization()(X3)
    X3 = L.LeakyReLU(alpha=0.2)(X3)    #l/8,b/8,512
    
    X4 = L.MaxPool2D((2, 2), strides=2)(X3)    #l/16,b/16,512
    X4 = L.Conv2D(1024, (3, 3), padding='same')(X4)
    X4 = L.BatchNormalization()(X4)
    X4 = L.LeakyReLU(alpha=0.2)(X4)
    X4 = L.Conv2D(1024, (3, 3), padding='same')(X4)
    X4 = L.BatchNormalization()(X4)
    X4 = L.LeakyReLU(alpha=0.2)(X4)    #l/16,b/16,1024
    
    X4 = L.Conv2DTranspose(512, (2, 2), strides=2)(X4)    #l/8,b/8,512
    X4 = L.Concatenate()([X3, X4])     #l/8,b/8,1024
    X4 = L.Conv2D(512, (3, 3), padding='same')(X4)
    X4 = L.BatchNormalization()(X4)
    X4 = L.Activation('relu')(X4)
    X4 = L.Conv2D(512, (3, 3), padding='same')(X4)
    X4 = L.BatchNormalization()(X4)
    X4 = L.Activation('relu')(X4)    #l/8,b/8,512
    
    X3 = L.Conv2DTranspose(256, (2, 2), strides=2)(X4)    #l/4,b.4,256
    X3 = L.Concatenate()([X2, X3])     #l/4,b/4,512
    X3 = L.Conv2D(256, (3, 3), padding='same')(X3)
    X3 = L.BatchNormalization()(X3)
    X3 = L.Activation('relu')(X3)
    X3 = L.Conv2D(256, (3, 3), padding='same')(X3)
    X3 = L.BatchNormalization()(X3)
    X3 = L.Activation('relu')(X3)    #l/4,b/4,256
    
    X2 = L.Conv2DTranspose(128, (2, 2), strides=2)(X3)    #l/2,b/2,128
    X2 = L.Concatenate()([X1, X2])     #l/2,b/2,256
    X2 = L.Conv2D(128, (3, 3), padding='same')(X2)
    X2 = L.BatchNormalization()(X2)
    X2 = L.Activation('relu')(X2)
    X2 = L.Conv2D(128, (3, 3), padding='same')(X2)
    X2 = L.BatchNormalization()(X2)
    X2 = L.Activation('relu')(X2)   #l/2,b/2,128
    
    X1 = L.Conv2DTranspose(64, (2, 2), strides=2)(X2)    #l,b,64
    X1 = L.Concatenate()([X0, X1])    #l,b,128
    X1 = L.Conv2D(64, (3, 3), padding='same')(X1)
    X1 = L.BatchNormalization()(X1)
    X1 = L.Activation('relu')(X1)
    X1 = L.Conv2D(64, (3, 3), padding='same')(X1)
    X1 = L.BatchNormalization()(X1)
    X1 = L.Activation('relu')(X1)    #l,b,64
    
    X0 = L.Conv2D(3, (1, 1), strides=1)(X1)     #l,b,3 
    model = Model(inputs=Xinpt, outputs=X0)
    return model

In [22]:
model = UNet()
model.compile('adam', loss='mean_squared_error', metrics=['mae', 'acc'])

In [ ]:
from IPython import display
for epoch in range(1, 20):
    display.clear_output(wait=True)
    print('Epoch:', epoch)
    model.fit(gray_train, train, batch_size=64, validation_data=(gray_test, test))
    sample_images(test, gray_test, 5)


Epoch: 1


In [ ]:
model.save('unet.hdf5')

In [ ]:
sagemaker_session = sagemaker.Session()

bucket = "recolorising"
prefix = "model"

role = sagemaker.get_execution_role()

In [ ]:
inputs = sagemaker_session.upload_data(path="unet.hdf5", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))